# Aspect Classification Experiments

Goal: assign each comment to an aspect (e.g., teaching skills, behaviour, knowledge, relevancy, general). We progress from interpretable baselines to transformers and few-shot LLMs with explainability and ethics considerations.

In [ ]:
import pandas as pd
from pathlib import Path

df = pd.read_excel(Path('data_feedback.xlsx'))
df.head()

## 1. Taxonomy setup and ethics
- Define a **glossary** of aspects with concise definitions to guide models and annotators.
- Check class frequencies; consider merging or few-shot support for rare labels.
- Ensure the taxonomy avoids value-laden terms that could encode bias.

In [ ]:
aspect_glossary = {
    'teaching skills': 'Clarity, preparation, and delivery of content',
    'behaviour': 'Politeness, supportiveness, and interpersonal conduct',
    'knowledge': 'Subject-matter expertise and depth',
    'relevancy': 'Alignment of course with learner needs',
    'general': 'Overall impressions not tied to a specific facet',
}
aspect_glossary

## 2. Baseline: One-vs-Rest Linear Classifier
Rationale: quick baseline with high interpretability via feature weights.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report

X_train, X_val, y_train, y_val = train_test_split(df['comments'], df['aspect'], test_size=0.2, stratify=df['aspect'], random_state=42)

tfidf_ovr = Pipeline([('tfidf', TfidfVectorizer(ngram_range=(1,2), min_df=1)), ('clf', OneVsRestClassifier(LogisticRegression(max_iter=300, class_weight='balanced')))])
tfidf_ovr.fit(X_train, y_train)
print(classification_report(y_val, tfidf_ovr.predict(X_val)))

### Feature-level explainability
Inspect the highest-weighted n-grams per class to validate alignment with glossary definitions.

In [ ]:
import numpy as np
vectorizer = tfidf_ovr.named_steps['tfidf']
clf = tfidf_ovr.named_steps['clf']
feature_names = np.array(vectorizer.get_feature_names_out())
for i, cls in enumerate(clf.classes_):
    coefs = clf.estimators_[i].coef_[0]
    top_indices = coefs.argsort()[-10:][::-1]
    print(cls, feature_names[top_indices])

## 3. Intermediate: Multilingual robustness with character n-grams
Rationale: character n-grams improve robustness to typos and mixed-language expressions.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer as CharTfidf
char_model = Pipeline([('tfidf', CharTfidf(analyzer='char', ngram_range=(3,5), min_df=1)), ('clf', OneVsRestClassifier(LogisticRegression(max_iter=300, class_weight='balanced')))])
char_model.fit(X_train, y_train)
print(classification_report(y_val, char_model.predict(X_val)))

## 4. Advanced: Transformer fine-tuning
Rationale: contextual understanding helps disambiguate overlapping aspects (e.g., teaching skills vs. knowledge).

In [ ]:
# Skeleton for transformer training
# from datasets import Dataset
# from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
# model_name = 'distilbert-base-uncased'
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# label2id = {lbl:i for i,lbl in enumerate(sorted(df['aspect'].unique()))}
# id2label = {i:lbl for lbl,i in label2id.items()}
# dataset = Dataset.from_pandas(df[['comments', 'aspect']])
# dataset = dataset.map(lambda x: {'labels': label2id[x['aspect']]}, remove_columns=['aspect'])
# dataset = dataset.map(lambda x: tokenizer(x['comments'], truncation=True), batched=True)
# model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label2id), id2label=id2label, label2id=label2id)
# args = TrainingArguments(output_dir='./aspect-model', evaluation_strategy='epoch', num_train_epochs=10, learning_rate=2e-5, per_device_train_batch_size=16, load_best_model_at_end=True)
# trainer = Trainer(model=model, args=args, train_dataset=dataset, eval_dataset=dataset, tokenizer=tokenizer)
# trainer.train()

### Explainable AI for aspects
- Use attention/IG/SHAP to highlight tokens that map to each aspect.
- Pair explanations with glossary definitions to validate semantic alignment.

## 5. Few-shot LLM prompting
Rationale: handle rare aspects and adapt quickly to new categories without retraining.
- Use in-context examples per aspect.
- Add self-consistency (vote across multiple samples) and enforce JSON schema for reliability.

In [ ]:
# Pseudocode placeholder
# def llm_aspect_predict(text):
#     prompt = 'You are an educational feedback classifier. Choose one aspect from the list and return JSON {label, rationale}. Aspects: teaching skills, behaviour, knowledge, relevancy, general.'
#     return {'label': 'teaching skills', 'rationale': 'mentions teaching clarity'}

## 6. Multi-label extension (forward-looking)
Rationale: comments can mention multiple facets. Use sigmoid outputs or set-generation LLM prompts.

## 7. Error analysis & ethics
- Confusion matrix to identify commonly confused aspects.
- Manually inspect mispredictions with explanations; refine glossary or prompts.
- Ethics: avoid amplifying stereotypes; communicate model uncertainty and intended use.